In [4]:
from mast import *
import pandas
import pickle

def fetch_K2_targets(target_type='planets',verbose=False):
    K2_targets = pandas.read_csv('data/overlap_'+target_type+'.csv')
    if verbose: print('K2 targets:',K2_targets.shape[0])
    return K2_targets

def cross_match(target_type='planets',short_cad_TICs=None,verbose=False,vals=[]):
    if short_cad_TICs == None:
        short_cad_TICs = fetch_short_cad(verbose=verbose)
    else:
        if verbose: print('Short cadence targets:',len(short_cad_TICs))
    K2_targets = fetch_K2_targets(target_type=target_type,verbose=verbose)
    rows_list = []
    count = 0
    for index, row in K2_targets.iterrows():
        RA = row['RA']
        Dec = row['DEC']
        epic = row['EPIC']
        sector = None
        for i in range(1,27):
            if row['S'+str(i)] == 1:
                sector = i
        try:
            tic = tic_from_coords((float(RA),float(Dec)))[0]
        except ValueError:
            print(RA,Dec)
            pass
        if tic in short_cad_TICs:
            val_dict = {'EPIC':epic,'TIC':tic,'RA':RA,'DEC':Dec,'sector':sector}
            for additional_val in vals:
                val_dict[additional_val] = row[additional_val]
            rows_list.append(val_dict)
        if (count % 50) == 0:
            with open('intermediate_bright.pickle','wb') as handle:
                pickle.dump(rows_list,handle,protocol=pickle.HIGHEST_PROTOCOL)
        count += 1
    matches = pandas.DataFrame(rows_list)
    return matches

In [5]:
crossmatch_all = pandas.read_csv('data/crossmatch_all.csv')
crossmatch_TIC_set = set(crossmatch_all['0'].tolist())

In [ ]:
matches = cross_match(target_type='bright',short_cad_TICs=crossmatch_TIC_set,verbose=False)

In [7]:
filename = 'data/crossmatch_bright.csv'
matches.to_csv('data/crossmatch_bright.csv')